In [1]:
import numpy as np 
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import os
#print(os.listdir("C:/Users/user/Downloads/captchas/images"))

In [2]:
import os
import os.path
import glob
import cv2
import random

In [3]:
#Finding all the unique letters of dataset
CAPTCHA_IMAGE_FOLDER ='C:/Users/user/Downloads/captchass/images'
# List of captchas.
captcha_images = glob.glob(os.path.join(CAPTCHA_IMAGE_FOLDER, "*"))
List_letters=[]
for (i, captcha_image) in enumerate(captcha_images):
    #print("Image {}/{}".format(i + 1, len(captcha_image)))

    # Get the letter labels from the imgae names.
    filename = os.path.basename(captcha_image)
    captcha_correct_text = os.path.splitext(filename)[0]
    List_letters.append(captcha_correct_text)

In [4]:
list2_letters=[]
for i in List_letters:
    for j in i:
        list2_letters.append(j)
unique=set(list2_letters)
unique_list=[]
for i in unique:
    unique_list.append(i)
unique_list.remove(' ')  
str1=''
str2=str1.join(unique_list)

In [5]:
str2

'EPKMDTBAGIHFYOWUXRCVQNJL'

In [13]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np

#Init main values
symbols = str2 # All symbols captcha can contain
num_symbols = len(symbols)
img_shape = (70, 200, 1)

Using TensorFlow backend.
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\AppData\Roaming

In [14]:
symbols

'EPKMDTBAGIHFYOWUXRCVQNJL'

In [15]:
print(num_symbols)

24


In [16]:
def create_model():
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 6 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(6):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    # Compile model and return it
    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [18]:
#from scipy import ndimage, misc
def preprocess_data():
    n_samples = len((os.listdir('C:/Users/user/Downloads/captchass/images'))) # see
    X = np.zeros((n_samples, 70, 200, 1)) #1070*50*200
    y = np.zeros((6, n_samples, num_symbols)) #5*1070*36

    #for i, pic in enumerate(os.listdir('C:/Users/user/Downloads/captchass/images')):
    allImages = os.listdir('C:/Users/user/Downloads/captchass/images')
    random.shuffle(allImages)
    for i in range(len(allImages)):
        # Read image as grayscale
        #print(i)
        img = cv2.imread(os.path.join('C:/Users/user/Downloads/captchass/images', allImages[i]))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # 
        pic_target = allImages[i][:-4]
        if len(pic_target) == 6:
            # Scale and reshape image
            img = img / 255.0
            #img=cv2.resize(img,(50,200),interpolation = cv2.INTER_AREA)
            img = np.reshape(img, (70, 200,1))
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((6, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
        
    # Return final data
    return X, y



In [19]:
X, y = preprocess_data() #

In [20]:
X.shape

(9836, 70, 200, 1)

In [21]:
y.shape   #....Y shape should be (9836, ...,...) how to do ??

(6, 9836, 24)

In [22]:
#from sklearn.utils import shuffle
#X,y=shuffle(X,y)

In [23]:
X_train, y_train = X[:8000], y[:, :8000] 
X_test, y_test = X[8000:], y[:, 8000:]

In [24]:
y.shape

(6, 9836, 24)

In [25]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
model=create_model();
model.summary();








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70, 200, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 70, 200, 16)  160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 35, 100, 16)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 35, 100, 32)  4640        max

In [27]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5]], batch_size=32, epochs=10,verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 6400 samples, validate on 1600 samples
Epoch 1/10
6400/6400 [==============================] - 209s 33ms/step - loss: 17.8597 - dense_2_loss: 2.8297 - dense_4_loss: 2.9935 - dense_6_loss: 2.9839 - dense_8_loss: 2.9241 - dense_10_loss: 3.0821 - dense_12_loss: 3.0464 - dense_2_acc: 0.0880 - dense_4_acc: 0.0667 - dense_6_acc: 0.0687 - dense_8_acc: 0.0761 - dense_10_acc: 0.0638 - dense_12_acc: 0.0634 - val_loss: 16.2056 - val_dense_2_loss: 2.2850 - val_dense_4_loss: 2.7027 - val_dense_6_loss: 2.9373 - val_dense_8_loss: 2.5749 - val_dense_10_loss: 2.9401 - val_dense_12_loss: 2.7656 - val_dense_2_acc: 0.2700 - val_dense_4_acc: 0.2025 - val_dense_6_acc: 0.1294 - val_dense_8_acc: 0.1594 - val_dense_10_acc: 0.0850 - val_dense_12_acc: 0.0762
Epoch 2/10
6400/6400 [==============================] - 85s 13ms/step - loss: 14.6881 - dense_2_loss: 1.7742 - dense_4_loss: 2.4960 - dense_6_loss: 2.5128 

In [28]:
# Pickling the trained model
import pickle
filename = 'Pickled_captcha_model'
pickle.dump(model, open(filename, 'wb'))

In [29]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(200,70))
    if img is not None:
        img = img / 255.0
        res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
        ans = np.reshape(res, (6, 24))
        l_ind = []
        probs = []
        for a in ans:
            l_ind.append(np.argmax(a))
            #probs.append(np.max(a))

        capt = ''
        for l in l_ind:
            capt += symbols[l]
        return capt#, sum(probs) / 6
    else:
        print("No Image in mentioned path");


In [30]:
score= model.evaluate(X_test,[y_test[0], y_test[1], y_test[2], y_test[3], y_test[4],y_test[5]],verbose=1)
print('Test Loss and accuracy:', score)

1836/1836 [==============================] - 17s 10ms/step
Test Loss and accuracy: [1.714431059905906, 0.05376142369838018, 0.0997042681591703, 0.17155320060181722, 0.3029514689298756, 0.6208752096348599, 0.46558550188484277, 0.9950980392156863, 0.9814814814814815, 0.9629629629629629, 0.9537037037037037, 0.8981481482780057, 0.933551198127223]


In [31]:
#model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print(predict('C:/Users/user/Downloads/captchass/images/CKHTUM.jpg'))


CKHTUM
